In [1]:
import requests
import json
import pandas as pd
import numpy as np
import time
from datetime import datetime, date
import logging
from typing import Tuple, List
from tqdm import tqdm
import os

In [2]:
# setup logging
logging.basicConfig(filename='social_cc_data.log', format='(%(process)d): %(asctime)s - %(levelname)s - %(message)s')

# create output directory
outdir = '../data/social_stats/'
if not os.path.exists(outdir):
    os.mkdir(outdir)

In [3]:
# set constant values
FROM_DATE = date(2017, 1, 1)
TO_DATE = date(2021, 12, 31)

DAYS_LIMIT = (TO_DATE - FROM_DATE).days
TO_TS = time.mktime(TO_DATE.timetuple())

# fetch historic price data
base_url = 'https://min-api.cryptocompare.com/data'
headers = {
    'Accepts': 'application/json',
    'Authorization': 'Apikey 2572b1c288c45f00c57f9a0dfa1a0da93afc454365cfaaf7ee17820f898fa7b7',
}

def get_social_data(coinId: int) -> Tuple[pd.DataFrame, bool]:
    success = True
    url = base_url + f'/social/coin/histo/day?coinId={coinId}&toTs={TO_TS}&limit={DAYS_LIMIT}'
    try:
        res = requests.get(url, headers=headers)
    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
    data = json.loads(res.text)
    # check success
    if data['Response'] != 'Success' or data['Message'] != '':
        msg = f'Unusual response for {coinId}: [{data["Response"]}: {data["Message"]}]'
        logging.warning(msg)
    if data['HasWarning'] == True:
        print(data)
        input('Unexpected Warning, press any key to continue')
    if not 'Data' in data:
        success = False
        return None, success
    else:
        df = pd.DataFrame(data['Data'])
        return df, success


In [4]:
# load coin IDs
with open('../data/coin_info.json') as f:
    coin_info = json.load(f)

coin_ids = {sym: int(coin['Id']) for sym, coin in coin_info.items()}
len(coin_ids)

7584

In [6]:
# get price history
start = 1
symbols_skipped = []

for symbol, id in tqdm(list(coin_ids.items())[start:]):
    df, success = get_social_data(id)
    if success == True:
        df.to_csv(outdir + f'{symbol}.csv')
    else:
        symbols_skipped.append(symbol)

# append skipped symbols to csv file
df_symbols_skipped = pd.DataFrame(symbols_skipped)
df_symbols_skipped.to_csv(outdir + 'symbols_skipped_social.csv', mode='a', header=False, index=False)

100%|██████████| 7583/7583 [49:16<00:00,  2.57it/s]   
